In [1]:
import numpy as np
import pandas as pd



In [2]:
seasons = list(range(2011,2020))

df = None

for s in seasons:
    path = './output/{}_boxscores.csv'.format(str(s))
    _df = pd.read_csv(path)
    
    if df is not None:
        df = pd.concat([df,_df],axis=0)
    else:
        df = _df
        
print(len(df))

df = df.drop(columns=['away_ranking','home_ranking'])


102617


In [3]:
all_teams = list(set(list(df['winning_name'].values)+list(df['losing_name'].values)))
print(all_teams[:15])

teams = []
for t in all_teams:
    if '\n\t\t\t' not in t:
        teams.append(t)
        
print(len(teams))
print(teams[:15])

['Chowan\n\t\t\t', 'Oregon Tech\n\t\t\t', 'Oklahoma Baptist\n\t\t\t', 'Trinity (TX)\n\t\t\t', 'Marietta\n\t\t\t', 'Cincinnati-Clermont\n\t\t\t', 'West Alabama\n\t\t\t', 'Detroit', 'USC', 'Alaska-Anchorage\n\t\t\t', 'Penn State-Mont Alto\n\t\t\t', 'Maine-Presque Isle\n\t\t\t', "St. Joseph's", 'Worcester State\n\t\t\t', 'Colorado Mesa\n\t\t\t']
355
['Detroit', 'USC', "St. Joseph's", 'Texas-Rio Grande Valley', 'Davidson', 'Norfolk State', 'VCU', 'St. Bonaventure', 'UNLV', 'South Carolina State', 'Kennesaw State', 'Fresno State', 'Miami (OH)', 'Texas Tech', 'College of Charleston']


In [4]:
def add_season(df):
    # datetime
    df['date'] = pd.to_datetime(df['date'])
    
    df['month'] = df['date'].copy().dt.month.astype(int)
    df['year'] = df['date'].copy().dt.year.astype(int)
    df['season'] = df['year'].copy()
    df['season'] = np.where(df['month']>9, df['season']+1, df['season'])
    
    df = df.drop(columns=['month','year'])
    
    return df

df = add_season(df)


## Features

Time span types
- EWA season
- RA season
- EWA last 5 seasons

Box score feats
- offensive rating
- defensive rating
- pct of points from FTs
- pct of points from 2Pts
- pct of points form 3Pts
- offensive rebound rate
- defensive rebound rate
- pace
- win pct
- opponent win pct
- opp ortg 
- opp drtg



In [6]:
# game ids
def add_ids(df):
    df['wn_copy'] = df['winning_name'].str.replace('\n\t\t\t','').str.replace(' ','').str.lower()
    df['ln_copy'] = df['losing_name'].str.replace('\n\t\t\t','').str.replace(' ','').str.lower()
    
    df['wn_copy2'] = df['wn_copy'].str[:3]
    df['wn_copy3'] = df['wn_copy'].str[-3:]
    
    df['ln_copy2'] = df['ln_copy'].str[:3]
    df['ln_copy3'] = df['ln_copy'].str[-3:]
    
    df['game_id'] = df['date'].dt.strftime('%m%d%y') + df['wn_copy2'] + df['wn_copy3'] + df['ln_copy2']  + df['ln_copy3']
    
#     old = len(df)
    df = df.drop_duplicates(subset=['game_id'])
#     new = len(df)
    # 53184
    
#     print("Successfully dropped {} duplicate box scores".format(old-new))
    
    df = df.drop(columns=['wn_copy2','ln_copy2','wn_copy3','ln_copy3'])
    
    df['team1_id'] = df['season'].astype(str)+df['wn_copy']
    df['team2_id'] = df['season'].astype(str)+df['ln_copy']
    
    df2 = df.copy()
    
    df = df.drop(columns=['team1_id'])
    df2 = df2.drop(columns=['team2_id'])
    df = df.rename(columns={'team2_id':'team_id'})
    df2 = df2.rename(columns={'team1_id':'team_id'})
    
    df = pd.concat([df,df2],axis=0)
    
    print(len(df))
    #106,368
    
    return df

df = add_ids(df)



106368


In [12]:
# two different types of groupbys

df.groupby(['team_id'])['home_offensive_rating','away_defensive_rating'].count().reset_index().sort_values(by='home_offensive_rating',ascending=False)



,team_id,home_offensive_rating,away_defensive_rating
4666,2018michigan,41,41
528,2011uconn,41,41
3647,2016villanova,40,40
4944,2018villanova,40,40
869,2012louisville,40,40
2335,2014uconn,40,40
3623,2016unc,40,40
2040,2014kentucky,40,40
556,2011vcu,40,40
3044,2015wisconsin,40,40


In [ ]:
print(list(df))